In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile, os, shutil

base_dir = "/content/drive/MyDrive/Thermal-Training"
extract_dir = os.path.join(base_dir, "raw_extracted")

#Reset folder if exists
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)
os.makedirs(extract_dir, exist_ok=True)

#Loop through all zips in Thermal_Datasets
zip_files = [f for f in os.listdir(base_dir) if f.lower().endswith(".zip")]

if not zip_files:
    print("⚠️ No zip files found in:", base_dir)
else:
    for z in zip_files:
        zip_path = os.path.join(base_dir, z)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"✅ Extracted: {z}")

print("📂 Extracted datasets are now in:", extract_dir)
print("Contents inside raw_extracted:", os.listdir(extract_dir))

✅ Extracted: Object Detection.v1i.darknet.zip
✅ Extracted: thermal_image.v1i.multiclass.zip
✅ Extracted: thermal_fire.v3i.darknet.zip
✅ Extracted: Thermal Human Detection.v1i.darknet.zip
📂 Extracted datasets are now in: /content/drive/MyDrive/Thermal-Training/raw_extracted
Contents inside raw_extracted: ['README.dataset.txt', 'README.roboflow.txt', 'test', 'train', 'valid']


In [3]:
import random
import os
import shutil

# Raw folder (adjust if your images are in test/val/train)
raw_dir = os.path.join(extract_dir, "train")   # <-- change if needed
cnn_dir = os.path.join("/content/drive/MyDrive/Thermal-Training/cnn_dataset")

# Reset CNN dataset folder
if os.path.exists(cnn_dir):
    shutil.rmtree(cnn_dir)

# Define all labels
labels = ["fire", "no_fire", "human", "object"]

for split in ["train", "val"]:
    for label in labels:
        os.makedirs(os.path.join(cnn_dir, split, label), exist_ok=True)

# Collect all images
all_images = [f for f in os.listdir(raw_dir) if f.lower().endswith(".jpg")]
random.shuffle(all_images)

# 80/20 split
split_idx = int(0.8 * len(all_images))
train_imgs = all_images[:split_idx]
val_imgs = all_images[split_idx:]

# Rule: decide label
def get_label(filename):
    filename_lower = filename.lower()
    if "fire" in filename_lower:
        return "fire"
    elif "human" in filename_lower:
        return "human"
    elif "object" in filename_lower:
        return "object"
    else:
        return "no_fire" # Default to no_fire if no specific label found

# Copy
for img in train_imgs:
    label = get_label(img)
    shutil.copy(os.path.join(raw_dir, img), os.path.join(cnn_dir, "train", label, img))

for img in val_imgs:
    label = get_label(img)
    shutil.copy(os.path.join(raw_dir, img), os.path.join(cnn_dir, "val", label, img))

print("✅ CNN dataset ready at:", cnn_dir)
for label in labels:
    print(f"Train {label.capitalize()}:", len(os.listdir(os.path.join(cnn_dir, "train", label))))
    print(f"Val {label.capitalize()}:", len(os.listdir(os.path.join(cnn_dir, "val", label))))

✅ CNN dataset ready at: /content/drive/MyDrive/Thermal-Training/cnn_dataset
Train Fire: 1959
Val Fire: 510
Train No_fire: 8077
Val No_fire: 2000
Train Human: 0
Val Human: 0
Train Object: 0
Val Object: 0


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size and batch
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(cnn_dir, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(cnn_dir, "val"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Build CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Found 10036 images belonging to 4 classes.
Found 2510 images belonging to 4 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 183s 552ms/step - accuracy: 0.0019 - loss: -6458367488.0000 - val_accuracy: 0.0000e+00 - val_loss: -301434994688.0000
Epoch 2/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 84s 269ms/step - accuracy: 0.0000e+00 - loss: -1696689422336.0000 - val_accuracy: 0.0000e+00 - val_loss: -14097549623296.0000
Epoch 3/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.0000e+00 - loss: -30174808637440.0000 - val_accuracy: 0.0000e+00 - val_loss: -110781047767040.0000
Epoch 4/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 81s 256ms/step - accuracy: 0.0000e+00 - loss: -175159805739008.0000 - val_accuracy: 0.0000e+00 - val_loss: -442020720017408.0000
Epoch 5/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 85s 272ms/step - accuracy: 0.0000e+00 - loss: -622657179484160.0000 - val_accuracy: 0.0000e+00 - val_loss: -1238112101466112.0000
Epoch 6/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 80s 256ms/step - accuracy: 0.0000e+00 - loss: -1654319933816832.0000 - val_accuracy: 0.0000e+00 - val_loss: -2770755343351808.000

In [11]:
import os

save_dir = "/content/drive/MyDrive/Thermal-Training/CNN-MODEL"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "cnn_model_fire_detection_v2.keras")
model.save(save_path)
print("✅ Model saved in modern Keras format at:", save_path)

✅ Model saved in modern Keras format at: /content/drive/MyDrive/Thermal-Training/CNN-MODEL/cnn_model_fire_detection_v2.keras


In [12]:
import os

thermal_datasets_path = "/content/drive/MyDrive/Thermal-Training/"
print("Contents of", thermal_datasets_path, ":")
try:
    for item in os.listdir(thermal_datasets_path):
        print(item)
except FileNotFoundError:
    print("The 'Thermal Datasets' folder was not found at:", thermal_datasets_path)

Contents of /content/drive/MyDrive/Thermal-Training/ :
Object Detection.v1i.darknet.zip
thermal_image.v1i.multiclass.zip
thermal_fire.v3i.darknet.zip
Thermal Human Detection.v1i.darknet.zip
raw_extracted
cnn_dataset
cnn_model_keras
CNN-MODEL


In [13]:
import os

cnn_dataset_path = "/content/drive/MyDrive/Thermal-Training/cnn_dataset"
labels = ["fire", "no_fire", "human", "object"] # Assuming these are still the intended labels

print(f"Contents of {os.path.join(cnn_dataset_path, 'train')}:")
try:
    train_fire_count = len(os.listdir(os.path.join(cnn_dataset_path, "train", "fire")))
    train_nofire_count = len(os.listdir(os.path.join(cnn_dataset_path, "train", "no_fire")))
    train_human_count = len(os.listdir(os.path.join(cnn_dataset_path, "train", "human")))
    train_object_count = len(os.listdir(os.path.join(cnn_dataset_path, "train", "object")))
    print(f"  Train Fire: {train_fire_count}")
    print(f"  Train No_Fire: {train_nofire_count}")
    print(f"  Train Human: {train_human_count}")
    print(f"  Train Object: {train_object_count}")

except FileNotFoundError as e:
    print(f"  Error listing train directory: {e}")

print(f"\nContents of {os.path.join(cnn_dataset_path, 'val')}:")
try:
    val_fire_count = len(os.listdir(os.path.join(cnn_dataset_path, "val", "fire")))
    val_nofire_count = len(os.listdir(os.path.join(cnn_dataset_path, "val", "no_fire")))
    val_human_count = len(os.listdir(os.path.join(cnn_dataset_path, "val", "human")))
    val_object_count = len(os.listdir(os.path.join(cnn_dataset_path, "val", "object")))
    print(f"  Val Fire: {val_fire_count}")
    print(f"  Val No_Fire: {val_nofire_count}")
    print(f"  Val Human: {val_human_count}")
    print(f"  Val Object: {val_object_count}")
except FileNotFoundError as e:
    print(f"  Error listing val directory: {e}")

Contents of /content/drive/MyDrive/Thermal-Training/cnn_dataset/train:
  Train Fire: 1959
  Train No_Fire: 8077
  Train Human: 0
  Train Object: 0

Contents of /content/drive/MyDrive/Thermal-Training/cnn_dataset/val:
  Val Fire: 510
  Val No_Fire: 2000
  Val Human: 0
  Val Object: 0


In [14]:
import shutil
import os

# Zip the cnn_dataset folder
cnn_dataset_path = "/content/drive/MyDrive/Thermal-Training/cnn_dataset"
output_zip_dataset = "/content/cnn_dataset.zip"

if os.path.exists(cnn_dataset_path):
    shutil.make_archive(output_zip_dataset.replace('.zip', ''), 'zip', cnn_dataset_path)
    print(f"✅ Zipped '{cnn_dataset_path}' to '{output_zip_dataset}'")
else:
    print(f"❌ Folder '{cnn_dataset_path}' not found.")

# Zip the cnn_model_keras folder
cnn_model_keras_path = "/content/drive/MyDrive/Thermal-Training/cnn_model_keras"
output_zip_model = "/content/cnn_model_keras.zip"

if os.path.exists(cnn_model_keras_path):
    shutil.make_archive(output_zip_model.replace('.zip', ''), 'zip', cnn_model_keras_path)
    print(f"✅ Zipped '{cnn_model_keras_path}' to '{output_zip_model}'")
else:
    print(f"❌ Folder '{cnn_model_keras_path}' not found.")

# You can also download the .keras file directly if it exists
keras_model_path = "/content/drive/MyDrive/Thermal-Training/cnn_model_keras/cnn_model_fire_detection_v2.keras"
if os.path.exists(keras_model_path):
    print(f"✅ Keras model file exists and can be downloaded: '{keras_model_path}'")
else:
    print(f"❌ Keras model file not found at: '{keras_model_path}'")

✅ Zipped '/content/drive/MyDrive/Thermal-Training/cnn_dataset' to '/content/cnn_dataset.zip'
✅ Zipped '/content/drive/MyDrive/Thermal-Training/cnn_model_keras' to '/content/cnn_model_keras.zip'
✅ Keras model file exists and can be downloaded: '/content/drive/MyDrive/Thermal-Training/cnn_model_keras/cnn_model_fire_detection_v2.keras'
